# Module import and definition of useful functions

In [106]:
import pandas as pd
import numpy as np

The following function is used to encode a given name (used to create an index with the brand's name)

In [107]:
import base64

def encode(text):
  btext = text.encode('utf-8')[:6]
  return base64.b64encode(btext).decode("utf-8") 

# Read the data and first setup

For now, the data are read directly in the directory where a copy of the spreadsheet has been put (in github directory). In the future, this will need to change

In [108]:
list_groups_wanted = ["Danone","L'Oréal","Ferrero"]

In [109]:
# Read the data and create associated dataframes
df = pd.read_excel('BDD consolidée - toutes les tables_v03.xlsx')#, dtype={'Groupe': str})

xls = pd.ExcelFile('BDD consolidée - toutes les tables_v03.xlsx')
df_companies_data = pd.read_excel(xls, 'Companies data')
df_companies_brands = pd.read_excel(xls, 'Companies brands')
df_companies_competitors = pd.read_excel(xls, 'Companies competitors')
df_global_score = pd.read_excel(xls, 'Global score display')
df_emissions_scope = pd.read_excel(xls, 'Emissions scopes description')
df_direct_complete_score = pd.read_excel(xls, 'Direct and complete score displ')
df_direct_commitment = pd.read_excel(xls, 'Direct and complete commitment ')
df_coeff_director = pd.read_excel(xls, 'Coeff directeur pour graph')
df_pos_cursors = pd.read_excel(xls, 'Positionnement des curseurs')
df_marques_by_cat = pd.read_excel(xls, 'Table Marques par catégorie')

# Create a dataframe that will contain all the variables
df_final = df_companies_data.copy(deep=True)
df_final = df_final[df_final["Group"].isin(list_groups_wanted)]
list_groups = list_groups_wanted

In [110]:
df_final

,Group,Sector,Logo,Country code,Turnover,Revenue (in bn €),Revenue year,Climate website,Comment,Global score,...,Cat 5 amount,Cat 5 name,Cat 6 amount,Cat 6 name,Wording interactive 1,Wording interactive 2,Wording interactive 3,Wording interactive 4,Wording interactive 5,Wording interactive 6
3,Danone,Food,NaN,FR,25 Mrd € (2019),25.0,2019.0,https://www.danone.com/fr/impact/planet/toward...,Très fortes réductions de toutes ses émissions...,6.0,...,0.25,Sc.3 - Use of product,0.05,Others,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.
5,Ferrero,Food,NaN,IT,11 Mrd € (2019),11.0,2019.0,https://www.ferrerosustainability.com/int/en/o...,"Des émissions toujours en hausse. Depuis 2010,...",2.0,...,0.25,Sc.3 - Use of product,0.05,Others,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.
18,L'Oréal,Hygiene & Beauty,NaN,FR,28 Mrd € (2020),28.0,2020.0,https://www.loreal.com/fr/nos-engagements/pour...,Très forte réduction de ses émissions propres ...,5.0,...,0.25,Sc.3 - Use of product,0.05,Others,n.a.,n.a.,n.a.,n.a.,n.a.,n.a.


## Pre-formatting 

The following code snipets reformat some of the previous dataframes such that there are easier to use for later transformation (i.e. set a nex index...)

### Coeff_director

In [111]:
# format the table so that a multi-index gets back the coefficient directly
df_coeff_director = df_coeff_director.drop(columns=["degre final"])
df_coeff_director = df_coeff_director.set_index(["Value type","Courbes"])
df_coeff_director.head(10)

Coeff directeur
Value type          Courbes                 
en valeur absolue   2°C              -0.0125
                    1,8°C            -0.0250
                    1,5°C            -0.0420
par unité de volume 2°C              -0.0200
                    1,8°C            -0.0400
                    1,5°C            -0.0672
par unité de valeur 2°C              -0.0700
                    1,8°C            -0.1000
                    1,5°C            -0.1500

### Cursor position

In [112]:
df_pos_cursors = df_pos_cursors.drop(columns=["Catégorie du secteur","Intervalle","Commentaire","level"]).rename(columns={"Value type - Grandeur utilisée (évolution annuelle CO2)":"unit",
                "Borne max (évolution annuelle CO2)":"max",
                "Borne min (évolution annuelle CO2)":"min"})

df_pos_cursors = df_pos_cursors.set_index(["unit","Score"])

df_pos_cursors.head(10)

max     min
unit                Score                
en valeur absolue   6.0   -0.0420 -0.0840
                    5.0   -0.0250 -0.0420
                    4.0   -0.0125 -0.0250
                    3.0    0.0000 -0.0125
                    2.0    0.0420  0.0000
                    99.0      NaN     NaN
                    1.0       NaN     NaN
par unité de volume 6.0   -0.0672 -0.1344
                    5.0   -0.0400 -0.0672
                    4.0   -0.0200 -0.0400

### Scores display: global, complete, direct

In [113]:
df_global_score = df_global_score.set_index("Global score")
df_global_score.head(10)

,Color Hex,Short label,Long label,Badge,Logo path
Global score,,,,,
1.0,820000.0,Undisclosed,Carbon footprint not disclosed,NaN,assets/frames/climate_score/Frame_Unrevealed.png
2.0,C00000,Highly insufficient,Increasing its carbon footprint,NaN,assets/frames/climate_score/Frame_TotallyInsuf...
3.0,FF8939,Insufficient,Stable ou slowly decreasing carbon footprint,NaN,assets/frames/climate_score/Frame_Insufficient...
4.0,FEC800,Partial,Carbon footprint partially decreasing,NaN,assets/frames/climate_score/Frame_Partial.png
5.0,8CDF41,Strong,Carbon footprint strongly decreasing,NaN,assets/frames/climate_score/Frame_Strong.png
6.0,0DB800,Very strong,Carbon footprint very strongly decreasing,NaN,assets/frames/climate_score/Frame_VeryStrong.png


In [114]:
df_direct_complete_score = df_direct_complete_score.set_index("Direct or complete score")
df_direct_complete_score.head(10)

,Color Hex,Short label,Long label
Direct or complete score,,,
1.0,820000.0,Not reported,Not disclosed
2.0,C00000,Towards +4°C,Increasing emissions. Heading towards +4°C.
3.0,FF8939,Between +2°C and +3°C,Stable ou decreasing too slowly. Not compatibl...
4.0,FEC800,2°C,Slightly decreasing. Compatible 2°C.
5.0,8CDF41,Well-below 2°C,Strongly decreasing. Compatible well-below 2°C
6.0,0DB800,1.5°C,Very strongly decreasing. Compatible 1.5°C.
99.0,C00000,n.a. (too recent),Measure only recent
n.a.,n.a.,n.a.,n.a.


In [115]:
df_direct_commitment = df_direct_commitment.set_index("Direct or complete score")
df_direct_commitment.head(10)

,Color Hex,Short label,Long label
Direct or complete score,,,
1.0,820000.0,Not reported,Not reported
2.0,C00000,Towards +4°C,Towards +4°C
3.0,FF8939,Between +2°C and +3°C,Ambition not compatible 2°C
4.0,FEC800,2°C,Ambition compatible 2°C
5.0,8CDF41,Well-below 2°C,Ambition compatible Well-below 2°C
6.0,0DB800,1.5°C,"Ambition compatible 1,5°C"
99.0,BFBFBF,n.a. (too recent),No commitment disclosed
n.a.,n.a.,n.a.,na.


# Brands: names, id and logo

Create the id of the company using the name

In [116]:
df_final['company_id'] = [encode(x) for x in df_final['Group']]

Get the logo of the logo of the company using the encoded name with the proper path 

In [117]:
df_final['brand_logo'] = df_final['company_id'].apply(lambda x: "assets/Pics/"+x+".png")

Do some sorting on the brands dataframe of the company to get the top 5 brands

In [118]:
top_5_brands_names=[]
top_5_brands_logos=[]

# Do a loop over the list of groups and filter the table df_top_brands on the group name
# Keep only the first 5 groups
# If less than 5 groups, print 'NaN' instead

for (k,group) in enumerate(list_groups):
    df_top_brands = df_companies_brands[df_companies_brands["Groupe"]==group].sort_values("Ordre")[:5]

    group_5_brand_name=[]
    group_5_brand_logo=[]
    
    for i in range(5):
        if i<len(df_top_brands):
            group_5_brand_name.append(df_top_brands["Marques"].iloc[i])
            group_5_brand_logo.append(df_top_brands["Logo"].iloc[i])
           
        else:
            group_5_brand_name.append('NaN')
            group_5_brand_logo.append('NaN')

    top_5_brands_names.append(group_5_brand_name)    
    top_5_brands_logos.append(group_5_brand_logo)    


df_final["top_brands"] = top_5_brands_names

# Competitors : names

In [119]:
# Just need to rename the column to match the column of final dataframe and do a merge
df_companies_competitors.rename(columns={"Groupe":"Group"}, inplace=True)
df_final = pd.merge(df_final, df_companies_competitors, how="left", on="Group")

In [120]:
# TODO !!!! so far not used in dataviz as fas as i know

## Actual Scores

### Global score

This code does a junction with the table of global score display to get relevant information. Careful, this assumes that the global score has a value (NaN values might create errors)

In [121]:
# This assumes that there is always a value for global score
df_final[["global_score_hexa_color_code","global_score_short_label",
          "global_score_logo_path"]] = df_final["Global score"].apply(
    lambda x: df_global_score.loc[x])[["Color Hex", "Short label","Logo path"]]

### Direct score

In [122]:
# This computes the various percentages depending on the scenarios along with the cursor level
C1_perc_scenarios = []
C1_direct_level = []
C1_reduc_per_year=[]
C1_final_value = []
for (k,group) in enumerate(list_groups):
    ini_date = df_final.loc[k,"C1 initial date"]
    fin_date = df_final.loc[k,"C1 final date"]
    
    if (ini_date != "n.a." and fin_date != "n.a."):
        diff_year = int(fin_date) - int(ini_date)
        
        if (df_final.loc[k,"C1 reduction"] != "n.a."):
            C1_final_value.append(100.0 + 100.0*df_final.loc[k,"C1 reduction"])
        
        
        # percentage per scenarios
        dim_perc_2 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C1 unit"], '2°C'])
        dim_perc_18 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C1 unit"], '1,8°C'])
        dim_perc_15 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C1 unit"], '1,5°C'])
        C1_perc_scenarios.append([100.0+dim_perc_2,100.0+dim_perc_18,100.0+dim_perc_15])
        
        # compute direct level of the cursor
        reduc_per_year = df_final.loc[k,"C1 reduction"] / diff_year
        min_inter = float(df_pos_cursors.loc[df_final.loc[k,"C1 unit"], df_final.loc[k,"C1 direct score"]]["min"])
        max_inter = float(df_pos_cursors.loc[df_final.loc[k,"C1 unit"], df_final.loc[k,"C1 direct score"]]["max"])
        C1_reduc_per_year.append(100.0*reduc_per_year)

        # Just to check some 'weird cases'
        if (reduc_per_year <min_inter or reduc_per_year>max_inter):
            print("problem with ",df_final.loc[k,"Group"], "; C1 score in DBB : ",df_final.loc[k,"C1 direct score"],
                 "but the annual reduction is: ", 100.0*reduc_per_year)
            
        
        if df_final.loc[k,"C1 direct score"] == 6.0 and reduc_per_year<min_inter:
            C1_direct_level.append(6.99) ## max level cursor (outside of interval)
        elif df_final.loc[k,"C1 direct score"] == 2.0 and reduc_per_year>max_inter:
            C1_direct_level.append(2.0) ## max level cursor (outside of interval)    
        else:
            # linear fit in the score interval
            C1_direct_level.append(df_final.loc[k,"C1 direct score"] + (reduc_per_year-max_inter)/(min_inter - max_inter))
    else:
        C1_perc_scenarios.append(["n.a.","n.a.","n.a."])
        C1_direct_level.append("n.a.")
        C1_reduc_per_year.append("n.a.")
        C1_final_value.append("n.a.")


df_final[["C1 2deg final","C1 1,8deg final","C1 1,5deg final"]] = C1_perc_scenarios
df_final["C1 reduc per year"] = C1_reduc_per_year
df_final["C1 direct level"] = C1_direct_level
df_final["C1 final value"] = C1_final_value


# This assumes that there is always a value for direct score and computes the display related variables
df_final[["direct_score_hexa_color_code","direct_score_short_label"]] = df_final["C1 direct score"].apply(lambda x: df_direct_complete_score.loc[x])[["Color Hex", "Short label"]]


problem with  Ferrero ; C1 score in DBB :  2.0 but the annual reduction is:  6.920748524024988
problem with  L'Oréal ; C1 score in DBB :  6.0 but the annual reduction is:  -9.913487699999997


### C2 columns

In [123]:
# This computes the various percentages depending on the scenarios along with the cursor level
C2_perc_scenarios = []
C2_direct_level = []
C2_reduc_per_year=[]
C2_final_value = []
for (k,group) in enumerate(list_groups):
    ini_date = df_final.loc[k,"C2 initial date"]
    fin_date = df_final.loc[k,"C2 final date"]
    
    if (ini_date != "n.a." and fin_date != "n.a."):
        diff_year = int(fin_date) - int(ini_date)
        
        if (df_final.loc[k,"C2 reduction"] != "n.a."):
            C2_final_value.append(100.0 + 100.0*df_final.loc[k,"C2 reduction"])
        
        
        # percentage per scenarios
        dim_perc_2 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C2 unit"], '2°C'])
        dim_perc_18 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C2 unit"], '1,8°C'])
        dim_perc_15 = float(100.0*diff_year*df_coeff_director.loc[df_final.loc[k,"C2 unit"], '1,5°C'])
        C2_perc_scenarios.append([100.0+dim_perc_2,100.0+dim_perc_18,100.0+dim_perc_15])
        
        # compute direct level of the cursor
        reduc_per_year = df_final.loc[k,"C2 reduction"] / diff_year
        min_inter = float(df_pos_cursors.loc[df_final.loc[k,"C2 unit"], df_final.loc[k,"C2 complete score"]]["min"])
        max_inter = float(df_pos_cursors.loc[df_final.loc[k,"C2 unit"], df_final.loc[k,"C2 complete score"]]["max"])
        C2_reduc_per_year.append(100.0*reduc_per_year)

        # Just to check some 'weird cases'
        if (reduc_per_year <min_inter or reduc_per_year>max_inter):
            print("problem with ",df_final.loc[k,"Group"], "; C2 score in DBB : ",df_final.loc[k,"C2 complete score"],
                 "but the annual reduction is: ", 100.0*reduc_per_year)
            
        
        if df_final.loc[k,"C2 complete score"] == 6.0 and reduc_per_year<min_inter:
            C2_direct_level.append(6.99) ## max level cursor (outside of interval)
        elif df_final.loc[k,"C2 complete score"] == 2.0 and reduc_per_year>max_inter:
            C2_direct_level.append(2.0) ## max level cursor (outside of interval)    
        else:
            # linear fit in the score interval
            C2_direct_level.append(df_final.loc[k,"C2 complete score"] + (reduc_per_year-max_inter)/(min_inter - max_inter))
    else:
        C2_perc_scenarios.append(["n.a.","n.a.","n.a."])
        C2_direct_level.append("n.a.")
        C2_reduc_per_year.append("n.a.")
        C2_final_value.append("n.a.")


df_final[["C2 2deg final","C2 1,8deg final","C2 1,5deg final"]] = C2_perc_scenarios
df_final["C2 reduc per year"] = C2_reduc_per_year
df_final["C2 complete level"] = C2_direct_level
df_final["C2 final value"] = C2_final_value

# This assumes that there is always a value for complete score and computes the display related variables
df_final[["complete_score_hexa_color_code","complete_score_short_label"]] = df_final["C2 complete score"].apply(lambda x: df_direct_complete_score.loc[x])[["Color Hex", "Short label"]]


# Commitments scores

## E1 columns

In [124]:
# This computes the various percentages depending on the scenarios along with the cursor level
E1_direct_level = []
E1_reduc_per_year=[]
E1_final_value = []

for (k,group) in enumerate(list_groups):

    ini_date = df_final.loc[k,"E1 initial date"]
    fin_date = df_final.loc[k,"E1 final date"]
    
    if (ini_date != "n.a." and fin_date != "n.a."):
        diff_year = int(fin_date) - int(ini_date)
        if (df_final.loc[k,"E1 reduction"] != "n.a."):
            E1_final_value.append(100.0 - float(df_final.loc[k,"E1 reduction"].split('%')[0]))
        
        
        # compute direct level of the cursor
        reduc_per_year = -float(df_final.loc[k,"E1 reduction"].split('%')[0]) / diff_year
        min_inter = 100.0*float(df_pos_cursors.loc[df_final.loc[k,"E1 unit"], df_final.loc[k,"E1 score commitment direct"]]["min"])
        max_inter = 100.0*float(df_pos_cursors.loc[df_final.loc[k,"E1 unit"], df_final.loc[k,"E1 score commitment direct"]]["max"])
        E1_reduc_per_year.append(100.0*reduc_per_year)

        # Just to check some 'weird cases'
        if (reduc_per_year <min_inter or reduc_per_year>max_inter):
            print("problem with ",df_final.loc[k,"Group"], "; E1 score in DBB : ",df_final.loc[k,"E1 score commitment direct"],
                 "but the annual reduction is: ", reduc_per_year,"; and interval is: ",min_inter,max_inter)
            
        
        if df_final.loc[k,"E1 score commitment direct"] == 6.0 and reduc_per_year<min_inter:
            E1_direct_level.append(6.99) ## max level cursor (outside of interval)
        elif df_final.loc[k,"E1 score commitment direct"] == 2.0 and reduc_per_year>max_inter:
            E1_direct_level.append(2.0) ## max level cursor (outside of interval)    
        else:
            # linear fit in the score interval
            E1_direct_level.append(df_final.loc[k,"E1 score commitment direct"] + 
                                   (reduc_per_year-max_inter)/(min_inter - max_inter))
         
    else:
        E1_direct_level.append("n.a.")
        E1_reduc_per_year.append("n.a.")
        E1_final_value.append("n.a.")


df_final["E1 reduc per year"] = E1_reduc_per_year
df_final["E1 direct level"] = E1_direct_level
df_final["E1 final value"] = E1_final_value


# This assumes that there is always a value for direct score and computes the display related variables
df_final[["direct_ambition_hexa_color_code","direct_ambition_long_label"]] = df_final["E1 score commitment direct"].apply(lambda x: df_direct_commitment.loc[x])[["Color Hex", "Short label"]]


problem with  L'Oréal ; E1 score in DBB :  6.0 but the annual reduction is:  -11.11111111111111 ; and interval is:  -8.4 -4.2


## E2 columns

In [125]:
# This computes the various percentages depending on the scenarios along with the cursor level
E2_direct_level = []
E2_reduc_per_year=[]
E2_final_value = []

for (k,group) in enumerate(list_groups):

    ini_date = df_final.loc[k,"E2 initial date"]
    fin_date = df_final.loc[k,"E2 final date"]
    
    if (ini_date != "n.a." and fin_date != "n.a."):
        diff_year = int(fin_date) - int(ini_date)
        if (df_final.loc[k,"E2 reduction"] != "n.a."):
            E2_final_value.append(100.0 - float(df_final.loc[k,"E2 reduction"].split('%')[0]))
        
        
        # compute direct level of the cursor
        reduc_per_year = -float(df_final.loc[k,"E2 reduction"].split('%')[0]) / diff_year
        min_inter = 100.0*float(df_pos_cursors.loc[df_final.loc[k,"E2 unit"], df_final.loc[k,"E2 score commitment"]]["min"])
        max_inter = 100.0*float(df_pos_cursors.loc[df_final.loc[k,"E2 unit"], df_final.loc[k,"E2 score commitment"]]["max"])
        E2_reduc_per_year.append(100.0*reduc_per_year)

        # Just to check some 'weird cases'
        if (reduc_per_year <min_inter or reduc_per_year>max_inter):
            print("problem with ",df_final.loc[k,"Group"], "; E2 score in DBB : ",df_final.loc[k,"E2 score commitment"],
                 "but the annual reduction is: ", reduc_per_year,"; and interval is: ",min_inter,max_inter)
            
        
        if df_final.loc[k,"E2 score commitment"] == 6.0 and reduc_per_year<min_inter:
            E2_direct_level.append(6.99) ## max level cursor (outside of interval)
        elif df_final.loc[k,"E2 score commitment"] == 2.0 and reduc_per_year>max_inter:
            E2_direct_level.append(2.0) ## max level cursor (outside of interval)    
        else:
            # linear fit in the score interval
            E2_direct_level.append(df_final.loc[k,"E2 score commitment"] + 
                                   (reduc_per_year-max_inter)/(min_inter - max_inter))
         
    else:
        E2_direct_level.append("n.a.")
        E2_reduc_per_year.append("n.a.")
        E2_final_value.append("n.a.")


df_final["E2 reduc per year"] = E2_reduc_per_year
df_final["E2 complete level"] = E2_direct_level
df_final["E2 final value"] = E2_final_value


# This assumes that there is always a value for direct score and computes the display related variables
df_final[["complete_ambition_hexa_color_code","complete_ambition_long_label"]] = df_final["E2 score commitment"].apply(lambda x: df_direct_commitment.loc[x])[["Color Hex", "Short label"]]


problem with  Danone ; E2 score in DBB :  5.0 but the annual reduction is:  -3.3333333333333335 ; and interval is:  -6.72 -4.0
problem with  Ferrero ; E2 score in DBB :  5.0 but the annual reduction is:  -3.5833333333333335 ; and interval is:  -6.72 -4.0
problem with  L'Oréal ; E2 score in DBB :  5.0 but the annual reduction is:  -3.5714285714285716 ; and interval is:  -6.72 -4.0


## Category

In [126]:
# First need to change the type to be a string otherwise it creates problems
df_final[["Cat 1 amount","Cat 2 amount","Cat 3 amount","Cat 4 amount","Cat 5 amount","Cat 6 amount"]] = df_final[["Cat 1 amount","Cat 2 amount","Cat 3 amount","Cat 4 amount","Cat 5 amount","Cat 6 amount"]].astype(str)


list_cat_names = []
list_cat_emissions = []
list_cat_hover = []

for (k,group) in enumerate(list_groups):    
    list_cat_names.append(",".join(list(df_final.loc[k,["Cat 1 name","Cat 2 name","Cat 3 name","Cat 4 name","Cat 5 name","Cat 6 name"]])))
    list_cat_emissions.append(",".join(list(df_final.loc[k,["Cat 1 amount","Cat 2 amount","Cat 3 amount","Cat 4 amount","Cat 5 amount","Cat 6 amount"]])))
    list_cat_hover.append(",".join(list(df_final.loc[k,["Wording interactive 1","Wording interactive 2","Wording interactive 3","Wording interactive 4","Wording interactive 5","Wording interactive 6"]])))

    
df_final["emissions_category_name"] = list_cat_names
df_final["emissions_category_amount"] = list_cat_emissions
df_final["emissions_category_hover"] = list_cat_names


# Renaming and final modifications

Change the name of some columns to match the expected names

In [128]:
df_data_viz = df_final.copy(deep=True)
df_data_viz = df_data_viz.rename(columns={
    "Group": "company_name",
    "Sector": "sector",
    "C1 final value": "c1_final_value",
    "C1 2deg final": "c1_2deg_final",
    "C1 1,8deg final": "c1_1_8deg_final",
    "C1 1,5deg final": "c1_1_5_deg_final",
    "C1 initial date": "C1_initial_date",
    "C1 final date": "C1_final_date",
    "C1 direct level": "direct_level",    
    "C2 final value": "c2_final_value",
    "C2 2deg final": "c2_2deg_final",
    "C2 1,8deg final": "c2_1_8deg_final",
    "C2 1,5deg final": "c2_1_5deg_final",
    "C2 initial date": "C2_initial_date",
    "C2 final date": "C2_final_date",
    "C2 complete level": "complete_level",
    "Comment": "comment",
    "Global score": "global_score",
    "C1 direct score": "direct_score",
    "C2 complete score": "complete_score",
    "E1 score commitment direct": "direct_rounding_score_commitments",
    "E1 direct level": "direct_score_commitments",
    "E1 phrase": "direct_commitments_sentence",
    "E2 score commitment": "complete_rounding_score_commitments",
    "E2 complete level": "complete_score_commitments",

    "E2 phrase": "complete_commitments_sentence",
    "Total emissions": "total_emissions",
    "Country code": "country_flag",
    
    "Revenue (in bn €)":"revenue",
    "Revenue year": "revenue_year",
    "Total emissions year":"total_emissions_year"
    
})

Complete list of variables

In [129]:

list_names_var_dataviz = ["company_name","c1_1_5_deg_final","c1_1_8deg_final","c1_2deg_final","C1_final_date",
                          "c1_final_value","C1_initial_date","c2_1_5deg_final","c2_1_8deg_final","c2_2deg_final",
                          "C2_final_date","c2_final_value","complete_score","global_score","complete_score_short_label",
                          "C2_initial_date","complete_score_hexa_color_code","direct_score","direct_score_hexa_color_code",
                          "complete_ambition_hexa_color_code","complete_rounding_score_commitments",
                          "direct_ambition_hexa_color_code","direct_rounding_score_commitments","direct_score_commitments",
                          "complete_score_commitments","comment","global_score_hexa_color_code","global_score_short_label",
                          "direct_score_short_label","global_score_logo_path","sector","complete_ambition_long_label",
                          "complete_commitments_sentence","direct_ambition_long_label","direct_commitments_sentence",
                          "emissions_category_amount","emissions_category_hover","emissions_category_name","total_emissions",
                          "total_emissions_year","brand_logo","company_id","country_flag","revenue","revenue_year","top_brands",
                          "direct_level","complete_level"]


In [130]:
df_data_viz = df_data_viz[list_names_var_dataviz]

,company_name,c1_1_5_deg_final,c1_1_8deg_final,c1_2deg_final,C1_final_date,c1_final_value,C1_initial_date,c2_1_5deg_final,c2_1_8deg_final,c2_2deg_final,...,total_emissions,total_emissions_year,brand_logo,company_id,country_flag,revenue,revenue_year,top_brands,direct_level,complete_level
0,Danone,79.0,87.5,93.75,2020,62.061092,2015,66.40,80.0,90.00,...,2327352.0,2019.0,assets/Pics/RGFub25l.png,RGFub25l,FR,25.0,2019.0,"[Activia, Danette, Michel et Augustin, Evian, ...",6.806615,5.551471
1,Ferrero,83.2,90.0,95.00,2019,127.682994,2015,87.40,92.5,96.25,...,2327352.0,2019.0,assets/Pics/RmVycmVy.png,RmVycmVy,IT,11.0,2019.0,"[Garnier, La Roche-Posay, Maybelline, Mixa, Na...",2.000000,2.404762
2,L'Oréal,87.4,92.5,96.25,2019,70.259537,2016,79.84,88.0,94.00,...,2327352.0,2019.0,assets/Pics/TCdPcsOp.png,TCdPcsOp,FR,28.0,2020.0,"[Nutella, Kinder, Ferrero Rocher, Mon Chéri, D...",6.990000,3.650000


In [134]:
df_data_viz[["company_name", "top_brands"]]

,company_name,top_brands
0,Danone,"[Activia, Danette, Michel et Augustin, Evian, ..."
1,Ferrero,"[Garnier, La Roche-Posay, Maybelline, Mixa, Na..."
2,L'Oréal,"[Nutella, Kinder, Ferrero Rocher, Mon Chéri, D..."


In [135]:
df_data_viz[["company_name", "company_id"]]

,company_name,company_id
0,Danone,RGFub25l
1,Ferrero,RmVycmVy
2,L'Oréal,TCdPcsOp


# Export files

In [ ]:
df_data_viz.to_excel("BDD_for_dataviz.xlsx") 

# Transform in proper types

Handle the types of all variables

In [69]:

'''
dict_type_of_variable = {
    "company_name": str,
    "c1_1_5_deg_final":float,
    "c1_1_8deg_final":float,
    "c1_2deg_final":float,
    "C1_final_date":str,
    "c1_final_value":float,
    "C1_initial_date":str,
    "c2_1_5deg_final":float,
    "c2_1_8deg_final":float,
    "c2_2deg_final":float,
    "C2_final_date":str,
    "c2_final_value":float,
    "complete_score":int,
    "global_score":int,
    "complete_score_short_label":str,
    "C2_initial_date":int,
    "complete_score_hexa_color_code":str,
    "direct_score":int,
    "direct_score_hexa_color_code":str,
    "complete_ambition_hexa_color_code":str,
    "complete_rounding_score_commitments":int,
    "direct_ambition_hexa_color_code":str,
    "direct_rounding_score_commitments":int,
    "direct_score_commitments":float,
    "complete_score_commitments":float,
    "comment":str,
    "global_score_hexa_color_code":str,
    "global_score_short_label":str,
    "direct_score_short_label":str,
    "global_score_logo_path":str,
    "sector":str,
    "complete_ambition_long_label":str,
    "complete_commitments_sentence":str,
    "direct_ambition_long_label":str,
    "direct_commitments_sentence":str,
    "emissions_category_amount":str,
    "emissions_category_hover":str,
    "emissions_category_name":str,
    "total_emissions":int,
    "total_emissions_year":int,
    "brand_logo":str,
    "company_id":str,
    "country_flag":str,
    "revenue":float,
    "revenue_year":int,
    "top_brands":list,
    "direct_level":float,
    "complete_level":float
}

# Change the type of the variable to match the expected type for the visualisation
df_data_viz = df_data_viz.astype(dict_type_of_variable, errors='raise')
'''